In [64]:
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
from plotly.subplots import make_subplots

Read the dataframe. Then, convert the date column to datetime format and the number of total engagements and number of tweets column to integer format.

In [65]:
df_news_articles_stats = pd.read_csv("news_articles_stats.csv")
df_news_articles_stats["Date"]= pd.to_datetime(df_news_articles_stats["Date"])
df_news_articles_stats["Total number of engagements"] = pd.to_numeric(df_news_articles_stats["Total number of engagements"])

df_number_of_tweets = pd.read_csv("number_of_tweets.csv")
df_number_of_tweets["Date"]= pd.to_datetime(df_number_of_tweets["Date"])
df_number_of_tweets["Number of tweets"] = pd.to_numeric(df_number_of_tweets["Number of tweets"])

Checking the data types.

In [66]:
df_news_articles_stats.info()
df_number_of_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 12 columns):
 #   Column                                 Non-Null Count  Dtype         
---  ------                                 --------------  -----         
 0   Date                                   6 non-null      datetime64[ns]
 1   News agency                            6 non-null      object        
 2   Article title                          6 non-null      object        
 3   Notes                                  1 non-null      object        
 4   Number of Facebook engagements         4 non-null      float64       
 5   Number of Twitter shares               4 non-null      float64       
 6   Number of Twitter retweets and linkes  1 non-null      float64       
 7   Number of Reddit engagements           4 non-null      float64       
 8   Youtube views                          1 non-null      float64       
 9   Total number of engagements            6 non-null      int64         

In [67]:
fig = go.Figure(data=[go.Scatter(
    x=df_news_articles_stats["Date"], 
    y = df_news_articles_stats["y"],
    mode='markers',
    marker=dict(
        size = df_news_articles_stats["Total number of engagements"],
        sizemode='area',
        sizeref=2.*max(df_news_articles_stats["Total number of engagements"])/(40.**2),
        sizemin=4
    )
)])

fig.show()

In [68]:
trace1 = go.Scatter(
    x=df_news_articles_stats["Date"], 
    y = df_news_articles_stats["y"],
    mode='markers',
    marker=dict(
        size = df_news_articles_stats["Total number of engagements"],
        sizemode='area',
        sizeref=2.*max(df_news_articles_stats["Total number of engagements"])/(40.**2),
        sizemin=4
    )
)


trace2 = go.Scatter(
    x=df_number_of_tweets["Date"], 
    y=df_number_of_tweets["Number of tweets"]
)

fig = go.Figure()
fig.add_trace(trace1)
fig.add_trace(trace2)
fig.show()